In [1]:
import  numpy as np
import pandas as pd
import requests
import math


In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
print(stocks)

    Ticker
0        A
1      AAL
2      AAP
3     AAPL
4     ABBV
..     ...
500    YUM
501    ZBH
502   ZBRA
503   ZION
504    ZTS

[505 rows x 1 columns]


In [3]:
from secret import IEX_CLOUD_API_TOKEN

### Making API  call
*  Market capitalization for each stock
* Price of each stock

In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 75992150, 'calculationPrice': 'close', 'change': -5.2, 'changePercent': -0.0342, 'close': 154.94, 'closeSource': 'afiifocl', 'closeTime': 1708798897009, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 156.041, 'delayedPriceTime': 1704176901654, 'extendedChange': -0.3, 'extendedChangePercent': -0.00208, 'extendedPrice': 153.47, 'extendedPriceTime': 1633700168683, 'high': 158.34, 'highSource': 'eelitumce1 pi 5rdeydna ', 'highTime': 1652966471816, 'iexAskPrice': None, 'iexAskSize': None, 'iexBidPrice': None, 'iexBidSize': None, 'iexClose': 152, 'iexCloseTime': 1710228283098, 'iexLastUpdated': None, 'iexMarketPercent': None, 'iexOpen': 153.8, 'iexOpenTime': 1659075346387, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexVolume': None, 'lastTradeTime': 1687052333756, 'latestPrice': 155.54, 'latestSource': 'Close', 'latestTime': 'September 10, 2021', 'latestUpdate': 1653506552811, 'latestVolume': 143893645, 'low': 154.6, 'lowSource': 'lm1iue  5 cytieer

In [5]:
# Parsing API call
price = data['latestPrice']
market_cap = data['marketCap']
print(market_cap / 1000000000000)

2.495912809765


In [6]:
# Create a pandas dataframes
my_columns = ['Ticker', 'Stock Price', 'Market Captialization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns=my_columns)

#Adding data on a dataframe
final_dataframe.append(
    pd.Series(
        [
            symbol,
            price,
            market_cap,
            'N/A'
        ],
    index = my_columns
    ),
    ignore_index=True
)

,Ticker,Stock Price,Market Captialization,Number of Shares to Buy
0,AAPL,155.54,2495912809765,N/A


In [7]:
# Looping through the Ticker in the list of stocks - single API requests
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks ['Ticker'][:10]:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
            [
                stock,
                data['latestPrice'],
                data['marketCap'], 
                'N/A'
            ],
            index= my_columns,
        ),
            ignore_index = True
    )
final_dataframe

,Ticker,Stock Price,Market Captialization,Number of Shares to Buy
0,A,185.18,55820041653,N/A
1,AAL,19.06,12461347697,N/A
2,AAP,209.92,12608106464,N/A
3,AAPL,150.58,2548346111145,N/A
4,ABBV,110.51,193468422973,N/A
5,ABC,123.41,25954750609,N/A
6,ABMD,378.90,17163780527,N/A
7,ABT,132.80,236899007602,N/A
8,ACN,358.98,236367794936,N/A
9,ADBE,659.20,325969301837,N/A


In [8]:
# Using batch API calls to 100 tickers
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [9]:
# Chunk generator of list
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range (0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

final_dataframe = pd.DataFrame(columns = my_columns)

# Data weighting from IEX Cloud.
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol, 
                    data[symbol]['quote']['latestPrice'], 
                    data[symbol]['quote']['marketCap'], 
                    'N/A'
                ], 
                index= my_columns
            ),
            ignore_index=True
        )
final_dataframe

,Ticker,Stock Price,Market Captialization,Number of Shares to Buy
0,A,182.25,53831800324,N/A
1,AAL,19.51,12532273268,N/A
2,AAP,203.92,13141458757,N/A
3,AAPL,151.55,2488463952242,N/A
4,ABBV,107.10,191039744485,N/A
...,...,...,...,...
500,YUM,130.36,38759155582,N/A
501,ZBH,146.94,29602605999,N/A
502,ZBRA,606.40,31865952274,N/A
503,ZION,59.68,9636986607,N/A


In [10]:
# Number of shares to buy
portfolio_size = float(input('Enter the value of your portfolio:'))

try:
    val = float(portfolio_size)
except ValueError: 
    print("That's not a number! \nPlease try again:") 
    portfolio_size = input('Enter the value of your portfolio:')
    val = float(portfolio_size)
portfolio_size

10000000.0

In [11]:
position_size = val / len(final_dataframe.index)
for i in range (0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i, 'Stock Price'])
final_dataframe

,Ticker,Stock Price,Market Captialization,Number of Shares to Buy
0,A,182.25,53831800324,108
1,AAL,19.51,12532273268,1014
2,AAP,203.92,13141458757,97
3,AAPL,151.55,2488463952242,130
4,ABBV,107.10,191039744485,184
...,...,...,...,...
500,YUM,130.36,38759155582,151
501,ZBH,146.94,29602605999,134
502,ZBRA,606.40,31865952274,32
503,ZION,59.68,9636986607,331


In [16]:
# Save data as a Excel Output
import xlsxwriter
writer = pd.ExcelWriter('recommended trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index=False)
print(final_dataframe)

    Ticker  Stock Price Market Captialization Number of Shares to Buy
0        A       182.25           53831800324                     108
1      AAL        19.51           12532273268                    1014
2      AAP       203.92           13141458757                      97
3     AAPL       151.55         2488463952242                     130
4     ABBV       107.10          191039744485                     184
..     ...          ...                   ...                     ...
500    YUM       130.36           38759155582                     151
501    ZBH       146.94           29602605999                     134
502   ZBRA       606.40           31865952274                      32
503   ZION        59.68            9636986607                     331
504    ZTS       212.00          100943200175                      93

[505 rows x 4 columns]


In [18]:
  # Create formats for .xlsx file
background_color = '#0a0a23',
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color':font_color, 
        'bg_color': background_color, 
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {   
        'num_format': '$0.00',
        'font_color':font_color, 
        'bg_color': background_color, 
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {   
        'num_format':'0',
        'font_color':font_color, 
        'bg_color': background_color, 
        'border': 1
    }
)

In [19]:
# Applying formats to columns in the xlsx file

column_formats = {
    'A': ['Ticker', string_format], 
    'B': ['Stock Price', dollar_format], 
    'C': ['Market Captializtion', dollar_format], 
    'D': ['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

print(writer)

In [21]:
# Keep getting Error right here
#writer.save()